In [1]:
import csv
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

def read_csv(root_path):
    with open(root_path + 'driving_log.csv', 'r') as f:
        reader = csv.reader(f)
        for line in reader:
            yield line

def make_local_path(path, local_root):
    basename = path.split('/')[-1]
    return local_root + 'IMG/' + basename

def load_image(local_path):
    return cv2.imread(local_path)

def extract_images(csv_line, local_root):
    paths = csv_line[0:3]
    return [load_image(make_local_path(p, local_root)) for p in paths]

def extract_measurements(csv_line):
    return float(csv_line[3])

def read_data(path):
    images = []
    measurements = []
    
    for line in read_csv(path):
        images.append(extract_images(line, path)[0])
        measurements.append(extract_measurements(line))
        
    return (np.array(images, np.float32), np.array(measurements, np.float32))

In [3]:
data_path = './driving/'

images, measurements = read_data(data_path)


In [5]:
np.save('X_train', images)
np.save('y_train', measurements)

In [6]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Convolution2D, MaxPooling2D, Lambda, Cropping2D, Dropout

image_shape = X_train.shape[1:]
normalize = lambda x: (x - 127.0) / 127.0
cropping = ((60, 0), (0, 0))
dropout_rate = 0.1

model = Sequential()
model.add(Lambda(normalize, input_shape=image_shape))
model.add(Cropping2D(cropping=cropping))
model.add(Convolution2D(24, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(36, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(48, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='relu'))
model.add(Convolution2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))
model.add(Convolution2D(64, kernel_size=(3, 3), padding='valid', activation='relu'))

model.add(Flatten())
model.add(Dropout(dropout_rate))
model.add(Dense(100, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(50, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, verbose=2, batch_size=256, epochs=8)

model.save('model.h5')

Train on 12728 samples, validate on 3183 samples
Epoch 1/8
22s - loss: 0.0553 - val_loss: 0.0487
Epoch 2/8
21s - loss: 0.0497 - val_loss: 0.0471
Epoch 3/8
21s - loss: 0.0492 - val_loss: 0.0475
Epoch 4/8
21s - loss: 0.0482 - val_loss: 0.0481
Epoch 5/8
21s - loss: 0.0480 - val_loss: 0.0475
Epoch 6/8
21s - loss: 0.0475 - val_loss: 0.0481
Epoch 7/8
21s - loss: 0.0471 - val_loss: 0.0469
Epoch 8/8
21s - loss: 0.0459 - val_loss: 0.0480


In [10]:
model.layers[4].output_shape

(None, 9, 37, 48)

In [ ]:
epochs = history.epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.ylim(0, 0.2)